In [ ]:
import undetected_chromedriver as uc
import re
from datetime import datetime
from lxml import html

In [ ]:
from scraping_tools import get_html, scrape_property
from parsing_tools import extract_thumbs

In [ ]:
url = 'https://www.aruodas.lt/butai-vilniuje-snipiskese-kalvariju-g-foto-yra-vizualizacija-parduodamas-geras-1-3308709/'
property_info = scrape_property(url) # works, nice

In [ ]:
property_info